In [21]:
from torch import nn, optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)
from torchvision.datasets import ImageFolder
from torchvision import transforms
import tqdm
import torch

In [22]:
# dataset
train_imgs=ImageFolder("taco_and_burrito/train/", transform=transforms.Compose([
    transforms.RandomCrop(224),transforms.ToTensor()
]))

test_imgs=ImageFolder("taco_and_burrito/test/", transform=transforms.Compose([
    transforms.CenterCrop(224), transforms.ToTensor()
]))

#DataLoader
train_loader= DataLoader(
    train_imgs, batch_size=32, shuffle=True)
test_loader= DataLoader(
    train_imgs, batch_size=32, shuffle=False)


In [23]:
print(train_imgs.classes)

['burrito', 'taco']


In [24]:
print(train_imgs.class_to_idx)

{'burrito': 0, 'taco': 1}


In [25]:
from torchvision import models

# 사전학습 된 resnet18 load
net=models.resnet18(pretrained=True)

# 모든 파라미터를 미분 대상에서 제외
for p in net.parameters():
    p.requires_grad=False
    
#마지막 선형 계층 변경
fc_input_dim= net.fc.in_features
net.fc=nn.Linear(fc_input_dim,2)

In [34]:
def eval_net(net,data_loader,device="cpu"):
    #dropout, batchNorm 무효화
    net.eval()
    ys=[]
    ypreds=[]
    for x, y in data_loader:
        x=x.to(device)
        y=y.to(device)
        
        # 확률이 가장 큰 분류를 예측하고 추론 계산이 전부이므로 자동미분에 필요한 처리를 off 설정해 불필요한 계산을 막음
        with torch.no_grad():
            _, y_pred=net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
        
    # 미니 배치 단위의 예측 결과를 하나로 묶음
    ys=torch.cat(ys)
    ypreds=torch.cat(ypreds)
    
    # 예측 정확도 계산
    acc=(ys==ypreds).float().sum()/len(ys)
    return acc.item()

def train_net(net,train_loader, test_loader, only_fc=True, 
              optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(), n_iter=10, device="cpu"):
    train_losses=[]
    train_acc=[]
    val_acc=[]
    if only_fc:
        #마지막 선형 계층의 파라미터만 optim에 전달
        optim=optimizer_cls(net.fc.parameters())
    else:
        optim=optimizer_cls(net.parameters())
        
    for e in range(n_iter):
        running_loss=0.0
        #신경망을 훈련 모드로 설정
        
        net.train()
        n=0
        n_acc=0
        
        for i,(xx,yy) in tqdm.tqdm(enumerate(train_loader),
                                  total=len(train_loader)):
                xx=xx.to(device)
                yy=yy.to(device)

                h=net(xx)
                loss=loss_fn(h,yy)
                optim.zero_grad()
                loss.backward()
                optim.step()

                running_loss += loss.item()
                n+=len(xx)
                _,y_pred=h.max(1)
                n_acc += (yy==y_pred).float().sum().item()
                
                #훈련 데이터의 예측 정확도
        train_losses.append(running_loss/i)
        train_acc.append(n_acc/n)
            
                #검증 데이터의 예측 정확도
        val_acc.append(eval_net(net,test_loader,device))
                #에포크 결과 표시
        print(e,train_losses[-1], train_acc[-1], val_acc[-1],flush=True)

In [35]:
# 신경망 파라미터 gpu전송
net.to("cuda:0")

train_net(net,train_loader,test_loader,n_iter=20,device="cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:03<00:00,  6.49it/s]


0 0.70616700026122 0.5786516853932584 0.6432584524154663


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  5.68it/s]


1 0.555159639228474 0.7542134831460674 0.8117977380752563


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:03<00:00,  5.79it/s]


...

17 0.3241460201415149 0.8820224719101124 0.8876404762268066


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  5.41it/s]


18 0.305034302175045 0.875 0.8862359523773193


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  4.71it/s]


19 0.32382800633257086 0.8679775280898876 0.8637640476226807


In [36]:
class FlattenLayer(nn.Module):
    def forward(self,x):
        sizes=x.size()
        return x.view(sizes[0],-1)
    
class IdentityLayer(nn.Module):
    def forward(self,x):
        return x
    
net=models.resnet18(pretrained=True)
for p in net.parameters():
    p.requires_grad=True
net.fc=IdentityLayer()

In [39]:
conv_net=nn.Sequential(
    nn.Conv2d(3,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    
    nn.Conv2d(64,128,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    FlattenLayer()
)

# 합성곱에 의해 어떤 크기인지 데이터를 넣어서 확인
test_input=torch.ones(1,3,224,224)
conv_output_size=conv_net(test_input).size()[-1]

#최종 CNN
net=nn.Sequential(
    conv_net,
    nn.Linear(conv_output_size,2)
)

net.to("cuda:0")

train_net(net,train_loader,test_loader,n_iter=10,only_fc=False,device="cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  4.94it/s]


0 4.457436770200729 0.5955056179775281 0.574438214302063


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  4.93it/s]


1 5.243909196420149 0.6123595505617978 0.574438214302063


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  5.02it/s]


...

7 4.545733088796789 0.6910112359550562 0.6797752976417542


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  4.52it/s]


8 4.555075751109556 0.6853932584269663 0.632022500038147


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  4.53it/s]


9 4.291507902586917 0.6839887640449438 0.6067416071891785
